## 1. Clustering

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import reuters
import gensim
import nltk
from gensim.test.utils import datapath
from gensim.models.doc2vec import TaggedLineDocument
from sklearn.cluster import KMeans
import pandas as pd

- Escolher um corpus e separar ele em sentencas formadas por tokens

In [2]:
def load_corpus():
    docs = []
    for fileid in reuters.fileids():
        docs.append(list(reuters.sents(fileid)))
    return docs

In [3]:
docs = load_corpus()
all_sents = [doc[0] for doc in docs]

with open('sentences.txt', 'w', encoding='utf8') as file:
    for sentence in all_sents:
        text_line = " ".join(sentence)
        file.write(f'{text_line}\n')

- Vetorizar as sentenças do documento. Teste as várias opções de vetorização que aprendemos (TF-IDF, Doc2Vec, LDA, etc).

In [4]:
%%time
# Criando o modelo
model = gensim.models.Doc2Vec(
    corpus_file='sentences.txt',
    vector_size=200,
    window=5,
    min_count=5,
    workers=8,
    epochs=100,
)

# Formatando entrada para o modelo e inserindo-a no modelo
tld = TaggedLineDocument("sentences.txt")
model.train(tld, total_examples=model.corpus_count, epochs=1)

# Criando o vetor de tokens
sents = []
with open('sentences{}.txt'.format(0), 'w', encoding='utf8') as file:
    for sentence in docs[0]:
        sents.append(sentence)
print("Exemplo de sentenca com os tokens (palavras) separados, que foi usado para treinar o modelo e que vai ser vetorizado na etapa seguinte:")
print(sents[0])
        
vetorzao = []
for i in sents:
    vector = model.infer_vector(i)
    vetorzao.append(vector)
print("\n\nExemplo de vetorizacao da sentenca acima, com 200 dimensoes:")
print(vetorzao[0])

Exemplo de sentenca com os tokens (palavras) separados, que foi usado para treinar o modelo e que vai ser vetorizado na etapa seguinte:
['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']


Exemplo de vetorizacao da sentenca acima, com 200 dimensoes:
[-5.42353955e-04  4.12977226e-02  4.10420373e-02 -5.72566539e-02
  1.46874199e-02 -2.35935976e-03  1.89877935e-02 -5.65520488e-02
  4.29281127e-03  4.94865440e-02 -3.71050760e-02  1.27204768e-02
  6.31043501e-03  8.26270133e-02  2.16809213e-02 -6.83984580e-03
  1.26413424e-02  2.04773583e-02  3.07334363e-02  3.05679794e-02
 -5.86936139e-02 -3.92802991e-02  3.80552411e-02 -6.67240750e-03
  2.5

- Agrupar as sentenças do documento em clusters usando K-Means.

In [6]:
%%time
kmeans_ = KMeans(n_clusters=6, random_state=42)
result = kmeans_.fit_transform(vetorzao)
print("Distancia de cada sentenca a cada cluster, respectivamente:")
print(result[0:5])
res = kmeans_.fit_predict(vetorzao)
print("\n\nQual 'categoria' se encaixa cada sentenca, qual o indice do cluster mais proximo da sentenca:")
print(res[0:5])
lista = [result[x][res[x]] for x in range(len(res))]
print("\n\nCom o indice do cluster mais proximo de cada sentenca, é possivel relacionar a distancia dessa sentenca ao seu cluster mais proximo, assim resultando na lista de sentencas mais proximas de seus clusters mais proximos a seguir:")
print(lista[0:5])

Distancia de cada sentenca a cada cluster, respectivamente:
[[4.18972918e-01 9.43791263e-01 1.16219615e+00 6.08483492e-01
  7.53776051e-01 1.03997684e+00]
 [9.09257071e-01 6.76677163e-01 1.62850313e+00 7.91070367e-01
  1.26591256e+00 1.04170832e+00]
 [6.45877696e-01 6.31479938e-01 1.31975458e+00 5.00145022e-01
  1.02801770e+00 9.83478731e-01]
 [1.02337094e+00 1.44240993e+00 3.65002415e-08 1.14167648e+00
  1.28812172e+00 1.37766441e+00]
 [8.37920020e-01 3.73051431e-01 1.42816536e+00 5.72514079e-01
  1.13763354e+00 8.04945473e-01]]


Qual 'categoria' se encaixa cada sentenca, qual o indice do cluster mais proximo da sentenca:
[0 1 3 2 1]


Com o indice do cluster mais proximo de cada sentenca, é possivel relacionar a distancia dessa sentenca ao seu cluster mais proximo, assim resultando na lista de sentencas mais proximas de seus clusters mais proximos a seguir:
[0.41897291775203066, 0.6766771630009544, 0.5001450221680627, 3.650024149988857e-08, 0.37305143097197735]
CPU times: user 68.4 

- Escolher as sentenças mais próximas do centro do cluster, para cada cluster.

In [9]:
final_list = []
for i in range(5):
    val = min(lista)
#     print(val)
    final_list.append(lista.index(val))
    del lista[lista.index(val)]

print("Escolhemos as 5 sentencas com a menor distancia a um cluster, assim assegurando que estas são as que melhor resumem o documento dado este modelo. Mesmo que duas sentencas pertencam a assuntos, ou clusteres, diferentes, se pegarmos as 5 com menor distancia, temos as 5 que melhor se relacionam a 5 dos 6 clusteres do modelo; descartando um dos clusteres, que nenhuma sentenca chegou tão perto.")
print()
print(sorted(final_list))



Escolhemos as 5 sentencas com a menor distancia a um cluster, assim assegurando que estas são as que melhor resumem o documento dado este modelo. Mesmo que duas sentencas pertencam a assuntos, ou clusteres, diferentes, se pegarmos as 5 com menor distancia, temos as 5 que melhor se relacionam a 5 dos 6 clusteres do modelo; descartando um dos clusteres, que nenhuma sentenca chegou tão perto.

[0, 2, 10, 11, 12]


- Exibir estas sentenças na ordem em que se apresentaram no texto original.

In [12]:
summary = []
for index in sorted(final_list):
    summary.append(" ".join(sents[index]))

print("Com as sentencas mais proximas dos clusteres listadas, faltou só ordena-las na ordem em que aparecem no documento original e assim é possivel resumir o texto original grosseiramente nas suas sentencas mais importantes, como vemos no paragrafo a seguir:")
print()
print(" ".join(summary))

Com as sentencas mais proximas dos clusteres listadas, faltou só ordena-las na ordem em que aparecem no documento original e assim é possivel resumir o texto original grosseiramente nas suas sentencas mais importantes, como vemos no paragrafo a seguir:

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain . Taiwan had a trade trade surplus of 15 . 6 billion dlrs last year , 95 pct of it with the U . S . The surplus helped swell Taiwan ' s foreign exchange reserves to 53 billion dlrs , among the world ' s largest . " We must quickly open our markets , remove trade barriers and cut import tariffs to allow imports of U . S . Products , if we want to d